In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


df = pd.read_csv("../input/dataset/train.tsv",sep="\t")
df.head()

In [ ]:
df.shape

In [ ]:
df = df.drop_duplicates(
    subset=['SentenceId'],
    keep='first'
)

In [ ]:
# set array dimensions
seq_len = 256
num_samples = len(df)

# initialize empty zero arrays
Xids = np.zeros((num_samples, seq_len))
Xmask = np.zeros((num_samples, seq_len))

# check shape
print(Xids.shape)
print(Xmask.shape)


In [ ]:
from transformers import BertTokenizer
# initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

for i, phrase in enumerate(df['Phrase']):
    tokens = tokenizer.encode_plus(phrase, max_length=seq_len, truncation=True,
                                   padding='max_length', add_special_tokens=True,
                                   return_tensors='tf')
    # assign tokenized outputs to respective rows in numpy arrays
    Xids[i, :] = tokens['input_ids']
    Xmask[i, :] = tokens['attention_mask']

In [ ]:
# first extract sentiment column
arr = df['Sentiment'].values

# we then initialize the zero array
labels = np.zeros((num_samples, arr.max()+1))

# set relevant index for each row to 1 (one-hot encode)
labels[np.arange(num_samples), arr] = 1

In [ ]:
import tensorflow as tf
# create the dataset object
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

def map_func(input_ids, masks, labels):
    # we convert our three-item tuple into a two-item tuple where the input item is a dictionary
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

# then we use the dataset map method to apply this transformation
dataset = dataset.map(map_func)

In [ ]:
# we will split into batches of 16
batch_size = 16

# shuffle and batch - dropping any remaining samples that don't cleanly
# fit into a batch of 16
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

In [ ]:
# set split size (90% training data) and calculate training set size
split = 0.9
size = int((Xids.shape[0]/batch_size)*split)

# get training and validation sets
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [ ]:
# AutoModel for PyTorch, TFAutoModel for TensorFlow
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('bert-base-cased')

In [ ]:
# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

# we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
embeddings = bert.bert(input_ids, attention_mask=mask)[1]  # access pooled activations with [1]

# convert bert embeddings into 5 output classes
x = tf.keras.layers.Dense(512, activation='relu')(embeddings)
y = tf.keras.layers.Dense(5, activation='softmax', name='outputs')(x)

In [ ]:
# initialize model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)


In [ ]:

optimizer = tf.keras.optimizers.Adam(lr=1e-5, decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=4
)

In [ ]:
model.save('sentiment_model')

In [ ]:
model = tf.keras.models.load_model('sentiment_model')

In [ ]:
# initialize tokenizer from transformers
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prep_data(text):
    # tokenize to get input IDs and attention mask tensors
    tokens = tokenizer.encode_plus(text, max_length=256,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_token_type_ids=False,
                                   return_tensors='tf')
    # tokenizer returns int32 tensors, we need to return float64, so we use tf.cast
    return {'input_ids': tf.cast(tokens['input_ids'], tf.float64),
            'attention_mask': tf.cast(tokens['attention_mask'], tf.float64)}

In [ ]:
in_tensor_negative = prep_data("The movie was awful")
in_tensor_positive = prep_data("The movie was brilliant")

probs_negative = model.predict(in_tensor_negative)[0]
probs_positive = model.predict(in_tensor_positive)[0]

print(probs_negative)
print(probs_positive)

In [ ]:

print("Negative:", np.argmax(probs_negative))
print("Positive:", np.argmax(probs_positive))